In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, DIAMETER_STD, TEMPERATURE_STD
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool
import os

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (depth - l23(_alpha) * np.sin(_alpha) - fh(_alpha)) / (1 - np.cos(_alpha))
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:83: RuntimeWarning: invalid value encountered in scalar multiply
  width / 2 - _r2 * np.sin(_alpha) - l23(_alpha) * np.cos(_alpha)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  - fw(_alpha) - (_r2 + r4) * np.sin(_alpha4)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (de

In [2]:
import pyroll.basic as pr
import pyroll.jmak

In [3]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_input_results.csv"

In [4]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

In [5]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

In [6]:
def worker(draw: DrawInput):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [7]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [01:57<00:00, 42.40it/s]


In [8]:
df = results |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                  roll_force                                \
      diameter  temperature             R1             R2             R3   
0     0.050390  1407.902098  413056.680811  237845.304357  436461.779330   
1     0.050772  1409.607889  419644.701447  240413.153440  436879.346306   
2     0.051325  1434.227183  408805.511930  233674.412698  421591.452552   
3     0.049996  1409.407579  403292.332396  233674.603534  433602.856642   
4     0.050787  1423.636210  407201.927846  234002.481319  426837.453203   
...        ...          ...            ...            ...            ...   
4995  0.049151  1422.545968  374447.347382  220295.098846  420436.404681   
4996  0.050889  1434.061562  400036.317949  230111.634726  419911.737924   
4997  0.048954  1429.411301  364838.714064  215614.702630  414741.628113   
4998  0.048303  1435.124846  347408.051867  207576.717810  407773.953459   
4999  0.050587  1422.700520  403879.618601  232713.380605  426652.479507   

                                                                  \
                 R4             R5             R6             R7   
0     185219.064531  261745.256743  140979.885477  264012.882321   
1     184732.789158  261100.261637  140652.986118  263504.571811   
2     178235.975729  252974.779925  136797.351459  257605.084402   
3     184837.438492  261320.723769  140803.956879  263755.799097   
4     181014.914801  256482.326731  138479.213216  260188.117903   
...             ...            ...            ...            ...   
4995  181390.670974  257140.172101  138897.285957  260873.480682   
4996  178302.300523  253099.710882  136880.434585  257742.620038   
4997  179597.425761  254933.775374  137873.543628  259316.418606   
4998  178121.429888  253170.277839  137091.725684  258154.285834   
4999  181272.664103  256823.642284  138651.813070  260456.817745   

                     ...    strain                                          \
                 R8  ...        R5        R6        R7        R8        R9   
0     119561.729117  ...  0.596819  0.499543  0.728239  0.596564  0.458585   
1     119369.767183  ...  0.596912  0.499568  0.728244  0.596561  0.458583   
2     117163.110498  ...  0.596978  0.499536  0.728210  0.596496  0.458538   
3     119468.175955  ...  0.596708  0.499506  0.728225  0.596557  0.458582   
4     118131.416724  ...  0.596878  0.499528  0.728218  0.596522  0.458557   
...             ...  ...       ...       ...       ...       ...       ...   
4995  118398.045215  ...  0.596421  0.499389  0.728176  0.596515  0.458556   
4996  117216.906161  ...  0.596875  0.499505  0.728201  0.596495  0.458538   
4997  117817.833009  ...  0.596339  0.499350  0.728157  0.596495  0.458543   
4998  117391.029299  ...  0.596103  0.499232  0.728114  0.596470  0.458529   
4999  118232.957802  ...  0.596830  0.499515  0.728215  0.596524  0.458558   

                                                        
           R10        F1        F2        F3        F4  
0     0.459897  0.342731  0.346689  0.447932  0.416565  
1     0.459892  0.342728  0.346686  0.447930  0.416562  
2     0.459834  0.342696  0.346650  0.447913  0.416549  
3     0.459894  0.342729  0.346688  0.447931  0.416563  
4     0.459859  0.342710  0.346666  0.447921  0.416544  
...        ...       ...       ...       ...       ...  
4995  0.459864  0.342713  0.346670  0.447923  0.416548  
4996  0.459835  0.342697  0.346650  0.447914  0.416549  
4997  0.459848  0.342704  0.346660  0.447919  0.416540  
4998  0.459834  0.342697  0.346653  0.447915  0.416552  
4999  0.459862  0.342711  0.346667  0.447922  0.416546  

[5000 rows x 100 columns]

In [9]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)